In [0]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score, f1_score, mean_absolute_error, mean_squared_error, confusion_matrix
from sklearn.naive_bayes import GaussianNB
from sklearn import svm
from matplotlib import pyplot as plt
import numpy as np
from google.colab.patches import cv2_imshow
import cv2

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [35]:
def getTestData():
  test_df=pd.read_csv("/content/gdrive/My Drive/features_test.csv")
  paths = test_df["path"].tolist()
  test_X=test_df.loc[:, ~test_df.columns.isin(['path'])].values
  print(test_X.shape)
  return paths,test_X

testFolder="/content/gdrive/My Drive/DatasetForTesting/"

test_paths,test_X = getTestData()

(10, 138)


In [12]:
df = pd.read_csv("/content/gdrive/My Drive/features_afterResizeOnly.csv")
print(df.shape)
df.head()

(3364, 143)


,height,label,path,width,partialWhitePixelCount_AfterImageResize,partialNonWhitePixelCount_AfterImageResize,ratio_PartialWhite_To_Total_AfterImageResize,ratio_PartialNoneWhite_To_Total_AfterImageResize,ratio_PartialNoneWhite_To_PartialWhite_AfterImageResize,completeWhitePixelCount_AfterImageResize,completeNonWhitePixelCount_AfterImageResize,ratio_CompleteWhite_To_Total_AfterImageResize,ratio_CompleteNoneWhite_To_Total_AfterImageResize,ratio_CompleteNoneWhite_To_CompleteWhite_AfterImageResize,PixelCountInRange_205_210_AfterImageResize,PixelCountInRange_210_215_AfterImageResize,PixelCountInRange_215_220_AfterImageResize,PixelCountInRange_220_225_AfterImageResize,PixelCountInRange_225_230_AfterImageResize,PixelCountInRange_230_235_AfterImageResize,PixelCountInRange_235_240_AfterImageResize,PixelCountInRange_240_245_AfterImageResize,PixelCountInRange_245_250_AfterImageResize,PixelCountInRange_250_255_AfterImageResize,blackToWhiteRatio_AfterImageResize,maxPossibleWhiteboardArea_AfterImageResize,maxPossibleWhiteboardAreaPercentage_AfterImageResize,HotspotPixelsCount_AfterImageResize,textAreaCount_AfterImageResize,textAreaPercentage_AfterImageResize,HotspotPixelsCount_Minval200_AfterImageResize,HotspotPixelsCount_Minval205_AfterImageResize,HotspotPixelsCount_Minval210_AfterImageResize,HotspotPixelsCount_Minval215_AfterImageResize,HotspotPixelsCount_Minval220_AfterImageResize,HotspotPixelsCount_Minval225_AfterImageResize,HotspotPixelsCount_Minval230_AfterImageResize,HotspotPixelsCount_Minval235_AfterImageResize,HotspotPixelsCount_Minval240_AfterImageResize,HotspotPixelsCount_Minval245_AfterImageResize,...,9Grids_ratio_PartialWhite_To_Total_AfterImageResize_Segment_3,9Grids_ratio_PartialNoneWhite_To_Total_AfterImageResize_Segment_3,9Grids_ratio_PartialNoneWhite_To_PartialWhite_AfterImageResize_Segment_3,9Grids_partialWhitePixelCount_AfterImageResize_Segment_4,9Grids_partialNonWhitePixelCount_AfterImageResize_Segment_4,9Grids_ratio_PartialWhite_To_Total_AfterImageResize_Segment_4,9Grids_ratio_PartialNoneWhite_To_Total_AfterImageResize_Segment_4,9Grids_ratio_PartialNoneWhite_To_PartialWhite_AfterImageResize_Segment_4,9Grids_partialWhitePixelCount_AfterImageResize_Segment_5,9Grids_partialNonWhitePixelCount_AfterImageResize_Segment_5,9Grids_ratio_PartialWhite_To_Total_AfterImageResize_Segment_5,9Grids_ratio_PartialNoneWhite_To_Total_AfterImageResize_Segment_5,9Grids_ratio_PartialNoneWhite_To_PartialWhite_AfterImageResize_Segment_5,9Grids_partialWhitePixelCount_AfterImageResize_Segment_6,9Grids_partialNonWhitePixelCount_AfterImageResize_Segment_6,9Grids_ratio_PartialWhite_To_Total_AfterImageResize_Segment_6,9Grids_ratio_PartialNoneWhite_To_Total_AfterImageResize_Segment_6,9Grids_ratio_PartialNoneWhite_To_PartialWhite_AfterImageResize_Segment_6,9Grids_partialWhitePixelCount_AfterImageResize_Segment_7,9Grids_partialNonWhitePixelCount_AfterImageResize_Segment_7,9Grids_ratio_PartialWhite_To_Total_AfterImageResize_Segment_7,9Grids_ratio_PartialNoneWhite_To_Total_AfterImageResize_Segment_7,9Grids_ratio_PartialNoneWhite_To_PartialWhite_AfterImageResize_Segment_7,9Grids_partialWhitePixelCount_AfterImageResize_Segment_8,9Grids_partialNonWhitePixelCount_AfterImageResize_Segment_8,9Grids_ratio_PartialWhite_To_Total_AfterImageResize_Segment_8,9Grids_ratio_PartialNoneWhite_To_Total_AfterImageResize_Segment_8,9Grids_ratio_PartialNoneWhite_To_PartialWhite_AfterImageResize_Segment_8,9Grids_HotspotPixelCount_AfterImageResize_Segment_0,9Grids_HotspotPixelCount_AfterImageResize_Segment_1,9Grids_HotspotPixelCount_AfterImageResize_Segment_2,9Grids_HotspotPixelCount_AfterImageResize_Segment_3,9Grids_HotspotPixelCount_AfterImageResize_Segment_4,9Grids_HotspotPixelCount_AfterImageResize_Segment_5,9Grids_HotspotPixelCount_AfterImageResize_Segment_6,9Grids_HotspotPixelCount_AfterImageResize_Segment_7,9Grids_HotspotPixelCount_AfterImageResize_Segment_8,RecognizedTextLength_AfterImageResize,NoOfLines_AfterImageResize,label_num
0,482.0,Whiteboard,00_27_37_00005.png,656

In [0]:
#df.columns.tolist()

In [0]:
#Removing highly correlated features
import numpy as np

# Create correlation matrix
corr_matrix = df.corr().abs()

# Select upper triangle of correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

# Find features with correlation greater than 0.95
to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]
# Drop features 
df.drop(to_drop, axis=1, inplace=True)

df.shape


(4022, 49)

In [14]:
X = df.loc[:, ~df.columns.isin(['path', 'label',"label_num","height","width"])].values
y = df["label_num"].values
print(X.shape,y.shape,type(y),type(X))
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)
print(X_train.shape,y_train.shape,type(X_train),type(y_train))

(3364, 138) (3364,) <class 'numpy.ndarray'> <class 'numpy.ndarray'>
(2523, 138) (2523,) <class 'numpy.ndarray'> <class 'numpy.ndarray'>


In [0]:
def metrics(y_true,y_pred):
  acc = accuracy_score(y_true,y_pred)
  print('Mean Absolute Error:', mean_absolute_error(y_true, y_pred))
  print('Mean Squared Error:', mean_squared_error(y_true, y_pred))
  print('Root Mean Squared Error:', np.sqrt(mean_squared_error(y_true, y_pred)))
  print("f1 score : ",f1_score(y_true, y_pred))
  print("Accuracy : ",acc)
  print("Confusion Matrix :",confusion_matrix(y_true, y_pred))


In [19]:
gnb = GaussianNB()
gnb.fit(X_train, y_train)
y_pred1 = gnb.predict(X_test)
metrics(y_test,y_pred1)

Mean Absolute Error: 0.21521997621878716
Mean Squared Error: 0.21521997621878716
Root Mean Squared Error: 0.4639180705887486
f1 score :  0.7454289732770746
Accuracy :  0.7847800237812128
Confusion Matrix : [[395  22]
 [159 265]]


In [20]:
clf = svm.LinearSVC()
clf.fit(X_train, y_train)
y_pred2 = clf.predict(X_test)
metrics(y_test,y_pred2)

Mean Absolute Error: 0.2437574316290131
Mean Squared Error: 0.2437574316290131
Root Mean Squared Error: 0.4937179676991846
f1 score :  0.6841294298921418
Accuracy :  0.7562425683709869
Confusion Matrix : [[414   3]
 [202 222]]


/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [26]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
rf.fit(X_train, y_train)
y_pred3 = rf.predict(X_test)
metrics(y_test,y_pred3)


Mean Absolute Error: 0.07015457788347206
Mean Squared Error: 0.07015457788347206
Root Mean Squared Error: 0.2648670947540899
f1 score :  0.9313154831199069
Accuracy :  0.929845422116528
Confusion Matrix : [[382  35]
 [ 24 400]]


In [22]:
from sklearn import tree
clf = tree.DecisionTreeClassifier()
clf.fit(X_train, y_train)
y_pred4 = clf.predict(X_test)
metrics(y_test,y_pred4)

Mean Absolute Error: 0.08204518430439953
Mean Squared Error: 0.08204518430439953
Root Mean Squared Error: 0.2864353056178647
f1 score :  0.9183431952662723
Accuracy :  0.9179548156956004
Confusion Matrix : [[384  33]
 [ 36 388]]


In [41]:
y_pred_test = rf.predict(test_X)
y_pred_test_prob = rf.predict_proba(test_X)
for i in range(len(test_paths)):
  img = cv2.imread(testFolder+test_paths[i])
  cv2_imshow(img)
  if(y_pred_test[i]==1):
    print("Whiteboard")
  else:
    print("Other")
  print(y_pred_test_prob[i])

Output hidden; open in https://colab.research.google.com to view.

In [33]:
cv2_imshow

NameError: ignored